In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import warnings
from datetime import *
warnings.simplefilter(action='ignore', category=FutureWarning)

In [61]:
#원하는 기간의 날짜를 리스트로 반환하는 함수
def date_range(start, end):
    '''
    startdate, enddate는 %Y%m%d 포맷으로 'YYYYMMDD' 형식
    '''
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [date.strftime("%Y.%m.%d") for date in pd.date_range(start, periods=(end-start).days+1)]
    return dates


def make_soup(url):
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    return soup




def last_botton_number(startdate,enddate,search_keyword,max_count_per_day):
    '''
    max_count_per_day = 크롤링할 일별 최대 뉴스의 개수
    
    
    네이버 뉴스페이지 구성

    시작페이지=1 step=10 으로 한 페이지당 10개의 뉴스를 노출시킨다.
    최대 노출 뉴스의 수는 400page 4000개의 뉴스를 노출시킨다. (그 이상은 불가능 필요하다면 검색 조건을 더 세부적으로 해야함)
    
    만약 검색한 키워드의 뉴스 총 노출수가 300개dlsep 100page를 url에 넣으면 알아서 300page로 들어가게된다

    따라서 마지막 페이지를 알아내려면 url에 page=4000을 넣으면 마지막 페이지로 이동할 수 있고
    그 soup에서 정규표현식을 통해 마지막 페이지의 숫자를 추출한다.



    날짜별로 반복:
        해당 날짜의 마지막 페이지 추출

        날짜, 마지막페이지번호, step 을 list에 추가

    return list
    '''

    url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_keyword}&sort=0&photo=0&field=0&pd=3&ds={date}&de={date}&cluster_rank=28&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{date_}to{date_},a:all&start={page}'

    #시작일 마감일 사이의 날짜들을 담은 리스트를 dates에 넣고 일 단위로 크롤링
    dates = date_range(startdate,enddate)
    
    date_lastpage_step = []

    for date in dates:  
        if max_count_per_day == -1:  
            url_ = url.format(search_keyword = search_keyword,
                            date = date,
                            date_ = date.replace('.',''),
                            page=4000)    # 네이버 뉴스페이지의 최대 노출 기사페이지 4000
            
            soup = make_soup(url_).select('a.btn') 
            
            try:    #검색 결과가 없는 경우 에러발생, 예외처리
                botton_count = int(re.findall('[0-9]+',str(soup[-1]).split(' ')[-1])[0]) * 10 - 9
                    
            except:
                print(date,'해당 일에 검색 결과가 존재하지 않습니다.')

        else:
            botton_count = max_count_per_day

        #페이지가 2이상 존재하지 않는 경우 step을 1로 설정
        step = 10 if botton_count > 10 else 1
        date_lastpage_step.append((date,botton_count,step))
    
    return date_lastpage_step



#['yna.co.kr', 'news.heraldcorp.com', 'gukjenews.com', 'view.asiae.co.kr', 'newspim.com', 'segye.com', 'news.kbs.co.kr','fnnews.com']
def main_text(url, news_agency):
    soup = make_soup(url)

    if news_agency == 'yna.co.kr':
        text = soup.select_one('#articleWrap > div.content01.scroll-article-zone01 > div > div > article').get_text()
        #result = [i[4:-4]  for i in str(text).split('\n') if i[:3]=='<p>']

    elif news_agency == 'yna.kr':
        if url.split('/')[3][:3] == 'AKR':
            text = soup.select_one('#articleWrap > div.content01.scroll-article-zone01 > div > div > article').get_text()
        else:
            text = soup.select_one('#viewWrap > div.inner-article > div.article-txt').get_text()

    elif news_agency == 'news.kbs.co.kr':
        text = soup.select_one('#cont_newstext').get_text()
        #result = [i for i in str(text).split('<br/>') if i != ''][1:]

    elif news_agency == 'fnnews.com':
        text = soup.select_one('#article_content').get_text().split('fn_get')[0]
        #result = list(filter(None,[i[5:] for i in str(text).split('\n') if i[:5] == '<br/>']))

    elif news_agency == 'segye.com':
        text = soup.select_one('#article_txt > article').get_text()
        #result = list(filter(None,[i[4:-4].replace('<br/>','') for i in str(text).split('\n') if i[:3] == '<p>']))[:-1]

    elif news_agency == 'view.asiae.co.kr':
        text = soup.select_one('#txt_area').get_text().split('(function(d,a,b,l,e,_)')[0]
        #result = [i[6:].replace('<span>','').replace('</span>','') if i[:5] == '<br/>' else i.split('</div> ')[-1] for i in ' '.join([i for i in str(text).split('<p>')]).split('</p>')]

    elif news_agency == 'news.heraldcorp.com':
        soup = make_soup(url)
        text = soup.select_one('#articleText').get_text()

    elif news_agency == 'newspim.com':
        text = soup.select_one('#news-contents').get_text()
        #result = [i[3:-4] for i in str(text).split('\n') if i[:3] == '<p>'] 

    elif news_agency == 'gukjenews.com':
        text = soup.select_one('#article-view-content-div').get_text()
        #result = [i[:-5].split('</p>')[0] if '</p>' in i[:-5] else i[:-5] for i in str(text).split('<p>')]













    return text



#start enddate default로 today()수정예정
def naver_news_url_crawling(search_keyword,max_count_per_day=-1,startdate='20220101',enddate='20220101'):
    '''

    max_count_per_day 하루 당 크롤링 할 최대 뉴스의 수 dafult시 (400,총 뉴스의 수) 중 작은값
    
    네이버 뉴스 웹은 page를 넘길 때 마다  (1, 11, 21, 31)순서로 바뀌고
    최대 page=4000 까지 지원한다. 페이지가 400이하일 때 page=4000을 넣으면 마지막 페이지가 나온다.
    매일 page=4000 url을 크롤링해서 마지막 페이지의 페이지버튼의 숫자를 통해 마지막 페이지를 크롤링하고
    for i in range(1, 총페이지수*10 -9, 10)으로 각 페이지마다 제목과 url 크롤링해서
    데이터프레임에 담아서 리턴 할 계획
    '''
    
    #검색하는 키워드의 띄어쓰기를 url상에 +으로 연결
    search_keyword = search_keyword.replace(' ','+')
    
    #네이버 뉴스 탭 url    
    url = 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search_keyword}&sort=0&photo=0&field=0&pd=3&ds={date}&de={date}&cluster_rank=28&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{date_}to{date_},a:all&start={page}'

    #빈 데이터프레임을 만들고 크롤링한 데이터를 1일 단위로 concat해서 반환 할 계획
    result = []
    
    

                                #last_botton_number 파라미터는 클래스 선언하면서 추후에 수정
                                #max_count_per_day -1일 경우 최대치로 크롤링 해야하는데 수정해야함
    for date, botton_count, step in last_botton_number(startdate,enddate,search_keyword,max_count_per_day):
        for page in range(1,botton_count+1,step): 
            url_ = url.format(search_keyword = search_keyword,
                              date = date,
                              date_= date.replace('.',''),
                              page = page)

            soup = make_soup(url_).select('a.news_tit')

            #한 페이지에 최대 노출 기사수 10개 1일 단위로 크롤링을 하니
            a_page_ulr = [html['href'] for html in soup[:min(10,max_count_per_day)]]
            a_page_title = [html['title'] for html in soup[:min(10,max_count_per_day)]]
            a_page_date = [date] * len(a_page_title)

            result.append(pd.DataFrame({'date':a_page_date, 'title':a_page_title, 'url':a_page_ulr}))
    
    result = pd.concat(result,ignore_index=True).drop_duplicates()

    # url에서 신문사의 정보를 유추할 수 있는 부분을 추출
    result['news_agency'] = result['url'].apply(lambda x : x.split('/')[2]).apply(lambda x : x.replace('www.',''))
    
    #main_text에서 선언한 언론사만 필터링
    news_agency = ['yna.co.kr', 'yna.kr','news.heraldcorp.com', 'gukjenews.com', 'view.asiae.co.kr', 'newspim.com', 'segye.com', 'news.kbs.co.kr','fnnews.com']
    result = result.loc[result['news_agency'].isin(news_agency)].reset_index(drop=True)

    #main_text
    result['main_text'] = [main_text(result.loc[i,'url'], result.loc[i,'news_agency']) for i in result.index]
    
    #특수문자제거
    result['main_text'] = result['main_text'].apply(lambda x : x.replace('\n','').replace('\t','').replace('\r',''))

    return result




In [ ]:
df = naver_news_url_crawling(search_keyword='교통',startdate='20220401',enddate='20220401',max_count_per_day=600)
df

In [37]:
#segye.com
url = 'https://www.yna.co.kr/view/PYH20220401042600013?input=1196m'
soup = make_soup(url)
text = soup.select_one('#viewWrap > div.inner-article > div.article-txt').get_text()
text

"\n\n\n\n\n\n\n임화영 기자\n기자 페이지\n\n\n\n (영종도=연합뉴스) 임화영 기자 = 인천공항 입국장 운영체계를 '코로나 이전' 수준으로 전환한 1일 오전 인천국제공항 1터미널 입국장에서 공항 관계자들이 지방자치단체 방역 안내소, 해외 입국 여행객 전용 대기·분리 장소 등 방역 관련 시설물을 철거하고 있다. \n 이날부터 해외 입국자는 어떤 국가에서 출발했는지와 상관없이 예방접종을 완료했다면 자가격리를 하지 않으며 대중교통을 이용할 수 있다. 2022.4.1\n hwayoung7@yna.co.kr\n"